In [237]:
import pandas as pd
import os
from IPython.display import display
from pandasql import sqldf
import numpy as np
import importlib

import plotly.express as px
import plotly.graph_objects as go
import maps_plotting_functions as mp
importlib.reload(mp)
dataset_path = 'dataset'
pysqldf = lambda q: sqldf(q, globals())


gold_df = pd.read_parquet(os.path.join(dataset_path, 'gold_partecipants.parquet'))
gold_df['population_display'] = gold_df['Population'].map('{:,}'.format)
gold_df = gold_df.assign(display= mp.get_display_customdata(gold_df))
display(gold_df.head(3))

,NOC,Year,events_partecipations_no,distinct_athletes_no,Bronze,Gold,NoMedal,Silver,TotalPartecipants,TotalMedals,region,alpha_3,iso_names,Population,population_display,display
0,AFG,1964,8,8,0,0,8,0,8,0,Afghanistan,AFG,Afghanistan,9744772,"9,744,772",<extra>AFG</extra><b>Afghanistan (1964)</b><br...
1,AFG,1968,5,5,0,0,5,0,5,0,Afghanistan,AFG,Afghanistan,10637064,"10,637,064",<extra>AFG</extra><b>Afghanistan (1968)</b><br...
2,AFG,1972,8,8,0,0,8,0,8,0,Afghanistan,AFG,Afghanistan,11791222,"11,791,222",<extra>AFG</extra><b>Afghanistan (1972)</b><br...


In [233]:

x_plot = np.linspace(1992, 2016, 50)

my_years = [2016, 2012, 2008]
gold_countries_derivatives_df = mp.add_worldwide_derivatives_to_df(gold_df)
gold_countries_derivatives_df.to_parquet(os.path.join(dataset_path, 'gold_countries_derivatives.parquet'))

In [234]:
# Get year 2016
def get_recent_three_years(df):
    years = df.Year.unique()
    years = np.sort(years)
    years = years[-3:]
    return np.flip(years)

recent_years = get_recent_three_years(gold_df)
print(recent_years)

[2016 2012 2008]


In [235]:
import pycountry

selected_country = "ESP"

if selected_country in gold_df.alpha_3.unique():
    fig = mp.get_fig_choropleth_similar_size(gold_df, selected_country, recent_years)
    fig.show(renderer='browser')
else:
    print(f"What did you mean by {selected_country}?")
    print(pycountry.countries.search_fuzzy(selected_country))


In [ ]:
# World View


In [236]:
fig = mp.get_fig_choropleth_world(gold_df, recent_years)
fig.show(renderer='browser')

In [ ]:
fig = mp.get_fig_choropleth_world(gold_countries_derivatives_df, recent_years, 'velocity')
fig.show(renderer='browser')

In [ ]:
fig = mp.get_fig_choropleth_world(gold_countries_derivatives_df, recent_years, 'acceleration')
fig.show(renderer='browser')

In [ ]:
gold_metrics_df = pd.read_parquet(os.path.join(dataset_path, 'gold_countries_year_metrics.parquet'))
gold_metrics_df['population_display'] = gold_metrics_df['Population'].map('{:,}'.format)
gold_metrics_df = gold_metrics_df.assign(display= mp.get_display_customdata(gold_metrics_df))

display(gold_metrics_df.head(3))


In [ ]:
fig = mp.get_fig_choropleth_world(gold_metrics_df, recent_years, 'AthletePerEventPartecipation')
fig.show(renderer='browser')


In [ ]:
fig = mp.get_fig_choropleth_world(gold_metrics_df, recent_years, 'PopulationPerMedal_thousands')
fig.show(renderer='browser')

In [ ]:
fig = mp.get_fig_choropleth_world(gold_metrics_df, recent_years, 'AthletePerMedal')
fig.show(renderer='browser')
